In [1]:
import pandas as pd
# import matplotlib.pyplot as plt

In [2]:
# Read in data
products = pd.read_csv("Resources/products.csv")
accounts = pd.read_csv("Resources/accounts.csv")
ast = pd.read_csv("Resources/account_state_transitions.csv")
gpa = pd.read_csv("Resources/group_product_associations.csv")
groups = pd.read_csv("Resources/groups.csv")
payments = pd.read_csv("Resources/payments.csv")

## 1) Generate a bar chart visualizing the relative frequency of Products registered with Accounts.

In [ ]:
gpa = gpa.rename(columns={"id":"gpa_id"})
accounts = accounts.rename(columns={"id":"accounts_df_id"})

In [ ]:
# Merge accounts and gpa data frames
group_id_merge = gpa.merge(accounts, how="inner",on="group_id")
group_id_merge.head()

In [ ]:
# Check w/ group by and manually
df_product_registration = pd.DataFrame(group_id_merge['product_id'].value_counts())
df_product_registration = df_product_registration.reset_index().sort_values('index')
df_product_registration = df_product_registration.rename(columns={"index":"product_id","product_id":"registration_count"})
df_product_registration = df_product_registration.reset_index(drop=True)
df_product_registration.head()

In [ ]:
# Make Bar Chart -- tuples?

## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

## 3) Devise a method of generating a table of all Accounts with their cumulative payments to date as of any arbitrary timestamp in the past.

In [4]:
payments.head()

,id,effective_when,account_id,currency,amount
0,1,2019-12-26 06:00:42.571392,1,KES,950
1,2,2019-12-26 06:00:42.571392,1,KES,950
2,3,2019-11-06 05:49:39.571392,2,KES,950
3,4,2019-11-06 05:49:39.571392,2,KES,950
4,5,2019-11-11 06:43:34.571392,2,KES,415


In [28]:
# Variable to take timestamp input.
timestamp = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2019-12-26


In [29]:
# Convert variable from string to datetime object
timestamp = pd.to_datetime(timestamp)
# print(timestamp)
# print(type(timestamp))

In [15]:
# Check data type of date in 'effective_when'
# type(payments['effective_when'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [16]:
# Change data type from string to datetime object
payments['effective_when'] = pd.to_datetime(payments['effective_when'])

In [17]:
# Check data type of date in 'effective_when'
# type(payments['effective_when'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [30]:
# Select rows containing indicated timestamp
a = payments.loc[payments['effective_when'] >= timestamp]
a.shape


# a = payments[payments['effective_when'].str.contains(timestamp,case=False)] #if string contains
# payments.sort_values('effective_when',ascending=False)

(4939, 5)

In [ ]:
# 2020-01-05 = 4389,5
# 2020-03-05 = 478,5
# 2019-01-05 = 9452, 5
# 2019-12-26 = 4939,5

In [ ]:
# SELECT all rows WHERE effect_when[i][:10] == timestamp
# group by account_id, sum currency for all rows >= DATE

emptylist = []

for item in payments['effective_when']: 
    date = item[:10]
    if date == timestamp:
        emptylist.append(date)
#         print(payments['account_id'])

In [ ]:
# Set currency
currency = payments.currency[0]

In [ ]:
# Group by account_id and count number of records
num_payments = payments.groupby('account_id').count()

# Flatten dataframe
num_payments = num_payments.rename_axis(None, axis=1).reset_index()

# Drop columns
num_payments = num_payments.drop(columns=['effective_when','currency','amount'])

# Rename column
cumulative_payments = num_payments.rename(columns={"id":"total_num_payments"})
cumulative_payments.head()

In [ ]:
# Group by account_id and sum payment amounts
sum_payments = payments.groupby('account_id').sum()

# Sort values by id
sum_payments = sum_payments.sort_values('id')

# Flatten dataframe
sum_payments = sum_payments.rename_axis(None, axis=1).reset_index()

# Select one column
sum_payments = sum_payments['amount']

# Add series to cumulative_payments dataframe
cumulative_payments[f'total_sum_payments_{currency}'] = sum_payments
cumulative_payments.head()

## 4) Devise a method of generating a table of all Accounts with their “nominal” expected payments as of any arbitrary timestamp in the past. In other words, we’re looking for the amount the distributor would have collected as of the test date assuming the client had paid the minimum payment instantaneously upon going to a “disabled” state until reaching the unlock price.